In [125]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

In [126]:
print(f"PyTorch version:{torch.__version__}") 

PyTorch version:1.10.2


In [127]:
# #M1 Mac GPU
# #Python 3.10 이상 & PyTorch 1.12.1 이상
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') #
# print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
# print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
# print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
# !python -c 'import platform;print(platform.platform())'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

### Hyperparameter Setting

In [128]:
CFG = {
    "IMG_SIZE" : 512,
    "EPOCHS" : 5,
    "LEARNING_RATE": 1e-4,
    "BATCH_SIZE": 16,
    "SEED" : 2
}

### Fixed RandomSeed

In [129]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(CFG["SEED"]) #Seed 고정

### Data Pre-processing

In [130]:
#Load Data
df_train = pd.read_csv("./data_original/data/train.csv")
df_test = pd.read_csv("./data_original/data/test.csv")

df_train["N_category"].value_counts(ascending = False)

1    514
0    486
Name: N_category, dtype: int64

In [131]:
train_pos_label_count = df_train["N_category"].value_counts(ascending = False)[0]
train_neg_label_count = df_train["N_category"].value_counts(ascending = False)[1]

#### 1. 암의 장경과 label

In [132]:
df_train[df_train["암의 장경"].isnull() & (df_train["N_category"] == 1)]
#임파선 전이를 하였지만 암의 장경이 결측값인 경우는 전체 데이터의 1% 미만인 4개 뿐임. >> 해당 데이터 제거

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
112,BC_01_0553,./train_imgs/BC_01_0553.png,-,40,2019-07-09,1,1,1,NaN,2.0,...,8.0,1.0,7.0,34.25,0.0,2.0,0.0,1.11,NaN,1
123,BC_01_0596,./train_imgs/BC_01_0596.png,-,49,2017-04-28,1,2,1,NaN,2.0,...,5.0,1.0,7.0,30.00,0.0,1.0,NaN,NaN,NaN,1
303,BC_01_1470,./train_imgs/BC_01_1470.png,-,37,2016-11-24,1,1,1,NaN,3.0,...,7.0,1.0,6.0,10.74,0.0,1.0,NaN,NaN,0.0,1
424,BC_01_2056,./train_imgs/BC_01_2056.png,-,61,2015-09-14,1,1,1,NaN,3.0,...,NaN,0.0,NaN,5.00,1.0,3.0,NaN,NaN,NaN,1


In [133]:
drop_row_idx_01 = df_train[df_train["암의 장경"].isnull() & (df_train["N_category"] == 1)].index
df_train.drop(drop_row_idx_01, axis = 0, inplace = True)
df_train.reset_index(drop = True, inplace = True)
len(df_train)

996

In [134]:
#결측치 보완
df_train["암의 장경"] = df_train["암의 장경"].fillna(df_train["암의 장경"].mean())
df_train = df_train.fillna(0)

df_test["암의 장경"] = df_test["암의 장경"].fillna(df_test["암의 장경"].mean())
df_test = df_test.fillna(0)

#### 2. Tubule formation (HG_score_1)과 label

In [135]:
df_train["HG_score_1"].value_counts()

3.0    617
2.0    206
0.0     90
1.0     83
Name: HG_score_1, dtype: int64

In [136]:
print(len(df_train[(df_train["HG_score_1"] == 1) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_1"] == 2) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_1"] == 3) & (df_train["N_category"] == 0)]) / train_neg_label_count)

print(len(df_train[(df_train["HG_score_1"] == 1) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_1"] == 2) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_1"] == 3) & (df_train["N_category"] == 1)]) / train_pos_label_count)

0.13618677042801555
0.20622568093385213
0.4396887159533074
0.026748971193415638
0.205761316872428
0.8045267489711934


In [137]:
drop_row_idx_02 = df_train[(df_train["HG_score_1"] == 1) & (df_train["N_category"] == 1)].index
df_train.drop(drop_row_idx_02, axis = 0, inplace = True)
df_train.reset_index(drop = True, inplace = True)
len(df_train)

983

#### 3. Nuclear Pleomorphism (HG_score_2)과 label

In [138]:
df_train["HG_score_2"].value_counts()

2.0    488
3.0    269
1.0    136
0.0     90
Name: HG_score_2, dtype: int64

In [139]:
print(len(df_train[(df_train["HG_score_2"] == 1) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_2"] == 2) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_2"] == 3) & (df_train["N_category"] == 0)]) / train_neg_label_count)

print(len(df_train[(df_train["HG_score_2"] == 1) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_2"] == 2) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_2"] == 3) & (df_train["N_category"] == 1)]) / train_pos_label_count)

0.22957198443579765
0.3754863813229572
0.17704280155642024
0.037037037037037035
0.6069958847736625
0.3662551440329218


In [140]:
drop_row_idx_02 = df_train[(df_train["HG_score_2"] == 1) & (df_train["N_category"] == 1)].index
df_train.drop(drop_row_idx_02, axis = 0, inplace = True)
df_train.reset_index(drop = True, inplace = True)
len(df_train)

965

#### 4. Mitotic Rate (HG_score_3)와 label

In [141]:
df_train["HG_score_3"].value_counts()

1.0    609
2.0    155
3.0    111
0.0     87
4.0      3
Name: HG_score_3, dtype: int64

In [142]:
print(len(df_train[(df_train["HG_score_3"] == 1) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_3"] == 2) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_3"] == 3) & (df_train["N_category"] == 0)]) / train_neg_label_count)
print(len(df_train[(df_train["HG_score_3"] == 4) & (df_train["N_category"] == 0)]) / train_neg_label_count)

print(len(df_train[(df_train["HG_score_3"] == 1) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_3"] == 2) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_3"] == 3) & (df_train["N_category"] == 1)]) / train_pos_label_count)
print(len(df_train[(df_train["HG_score_3"] == 4) & (df_train["N_category"] == 1)]) / train_neg_label_count)

0.5836575875486382
0.10700389105058365
0.0914396887159533
0.005836575875486381
0.6358024691358025
0.205761316872428
0.13168724279835392
0.0


In [143]:
drop_row_idx_02 = df_train[df_train["HG_score_3"] == 4].index
df_train.drop(drop_row_idx_02, axis = 0, inplace = True)
df_train.reset_index(drop = True, inplace = True)
len(df_train)

962

In [144]:
#Train / Validation Split
df_train, df_val, train_labels, val_labels = train_test_split(df_train.drop(columns = ["N_category"]),
                                                              df_train["N_category"],
                                                              test_size = 0.2,
                                                              random_state = CFG["SEED"])

In [145]:
#Numeric Feature Scaling / Categorical Feature Label-Encoding

def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ["나이", "암의 장경", "ER_Allred_score", "PR_Allred_score", "KI-67_LI_percent", "HER2_SISH_ratio"]
ignore_cols = ["ID", "img_path", "mask_path", "수술연월일", "N_category"]

for col in df_train.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        df_train[col] = scaler.fit_transform(get_values(df_train[col]))
        df_val[col] = scaler.transform(get_values(df_val[col]))
        df_test[col] = scaler.transform(get_values(df_test[col]))
    else:
        le = LabelEncoder()
        df_train[col] = le.fit_transform(get_values(df_train[col]))
        df_val[col] = le.transform(get_values(df_val[col]))
        df_test[col] = le.transform(get_values(df_test[col]))

### CustomDataset

In [146]:
class CustomDataset(Dataset):
    def __init__(self, df_medical, labels, transforms = None):
        self.df_medical = df_medical
        self.transforms = transforms
        self.labels = labels
    
    def __getitem__(self, index):
        img_path_prefix = "./data_original/data"
        img_path = img_path_prefix + self.df_medical["img_path"].iloc[index][1:]
        print(img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image = image)["image"]
            
        if self.labels is not None:
            tabular = torch.Tensor(self.df_medical.drop(columns = ["ID", "img_path", "mask_path", "수술연월일"]).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.df_medical.drop(columns = ["ID", "img_path", "수술연월일"]).iloc[index])
            return image, tabular
        
    def __len__(self):
        return len(self.df_medical)

In [147]:
img_path_prefix = "./data_original/data"
img_path = img_path_prefix + df_train["img_path"].iloc[1][1:]
img_path

'./data_original/data/train_imgs/BC_01_2821.png'

In [148]:
train_transforms = A.Compose([
                A.HorizontalFlip(),
                A.VerticalFlip(),
                A.Rotate(limit = 90, border_mode = cv2.BORDER_CONSTANT, p = 0.3),
                A.Resize(CFG["IMG_SIZE"], CFG['IMG_SIZE']),
                A.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), max_pixel_value = 255.0, always_apply = False, p = 1.0),
    ToTensorV2()
    ])

test_transforms = A.Compose([
    A.Resize(CFG["IMG_SIZE"], CFG['IMG_SIZE']),
    A.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225), max_pixel_value = 255.0, always_apply = False, p = 1.0),
    ToTensorV2()
])

In [149]:
train_dataset = CustomDataset(df_train, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG["BATCH_SIZE"], shuffle = True, num_workers = 0)

val_dataset = CustomDataset(df_val, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size = CFG["BATCH_SIZE"], shuffle = False, num_workers = 0)

### Model Architecture

In [150]:
class ImgFeatureExtraction(nn.Module):
    def __init__(self):
        super(ImgFeatureExtraction, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained = True)
        self.embedding = nn.Linear(1000, 512)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.embedding(x)
        return x

In [151]:
class TabularFeatureExtractor(nn.Module):
    def __init__(self):
        super(TabularFeatureExtractor, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(in_features = 23, out_features = 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features = 128, out_features = 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features = 256, out_features = 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features = 512, out_features = 512)
        )
    
    def forward(self, x):
        x = self.embedding(x)
        return x

In [152]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtraction()
        self.tabular_feature_extractor = TabularFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features = 1024, out_features = 1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, tabular):
        img_feature = self.img_feature_extractor(img)
        tabular_feature = self.tabular_feature_extractor(tabular)
        feature = torch.cat([img_feature, tabular_feature], dim = -1)
        output = self.classifier(feature)
        return output

### Train

In [153]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG["EPOCHS"] + 1):
        model.train()
        train_loss = []
        for img, tabular, label in tqdm(iter(train_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1, 1))
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
        val_loss, val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_score = val_score
            best_model = model
    
    return best_model


def validation(model, criterion, val_loader, device):
    #model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.5
    with torch.no_grad():
        model.eval()
        for img, tabular, label in tqdm(iter(val_loader)):
            true_labels += label.tolist()
            
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1, 1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_score = metrics.f1_score(y_true = true_labels, y_pred = pred_labels, average = "macro")
        return np.mean(val_loss), val_score

### Run

In [154]:
model = nn.DataParallel(ClassificationModel())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "max", factor = 0.5, patience = 1, threshold_mode = "abs", min_lr = 1e-8, verbose = True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/49 [00:00<?, ?it/s]

./data_original/data/train_imgs/BC_01_1354.png
./data_original/data/train_imgs/BC_01_2861.png
./data_original/data/train_imgs/BC_01_0506.png
./data_original/data/train_imgs/BC_01_2626.png
./data_original/data/train_imgs/BC_01_3139.png
./data_original/data/train_imgs/BC_01_1276.png
./data_original/data/train_imgs/BC_01_2631.png
./data_original/data/train_imgs/BC_01_1527.png
./data_original/data/train_imgs/BC_01_2528.png
./data_original/data/train_imgs/BC_01_2623.png
./data_original/data/train_imgs/BC_01_2743.png
./data_original/data/train_imgs/BC_01_0572.png
./data_original/data/train_imgs/BC_01_2687.png
./data_original/data/train_imgs/BC_01_0005.png
./data_original/data/train_imgs/BC_01_3247.png
./data_original/data/train_imgs/BC_01_2756.png


  2%|▏         | 1/49 [01:15<1:00:26, 75.55s/it]

./data_original/data/train_imgs/BC_01_2753.png
./data_original/data/train_imgs/BC_01_0730.png
./data_original/data/train_imgs/BC_01_1826.png
./data_original/data/train_imgs/BC_01_0257.png
./data_original/data/train_imgs/BC_01_2673.png
./data_original/data/train_imgs/BC_01_0295.png
./data_original/data/train_imgs/BC_01_1442.png
./data_original/data/train_imgs/BC_01_3045.png
./data_original/data/train_imgs/BC_01_1190.png
./data_original/data/train_imgs/BC_01_1662.png
./data_original/data/train_imgs/BC_01_3014.png
./data_original/data/train_imgs/BC_01_2106.png
./data_original/data/train_imgs/BC_01_1282.png
./data_original/data/train_imgs/BC_01_1101.png
./data_original/data/train_imgs/BC_01_2930.png
./data_original/data/train_imgs/BC_01_2734.png


In [ ]:
test_dataset = CustomDataset(df_test, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.5
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/16 [00:00<?, ?it/s]

./data_original/data/test_imgs/BC_01_0011.png
./data_original/data/test_imgs/BC_01_0220.png
./data_original/data/test_imgs/BC_01_0233.png
./data_original/data/test_imgs/BC_01_0258.png
./data_original/data/test_imgs/BC_01_0260.png
./data_original/data/test_imgs/BC_01_0266.png
./data_original/data/test_imgs/BC_01_0269.png
./data_original/data/test_imgs/BC_01_0272.png
./data_original/data/test_imgs/BC_01_0275.png
./data_original/data/test_imgs/BC_01_0278.png
./data_original/data/test_imgs/BC_01_0279.png
./data_original/data/test_imgs/BC_01_0286.png
./data_original/data/test_imgs/BC_01_0296.png
./data_original/data/test_imgs/BC_01_0303.png
./data_original/data/test_imgs/BC_01_0310.png
./data_original/data/test_imgs/BC_01_0322.png


  6%|▋         | 1/16 [00:08<02:12,  8.81s/it]

./data_original/data/test_imgs/BC_01_0340.png
./data_original/data/test_imgs/BC_01_0344.png
./data_original/data/test_imgs/BC_01_0390.png
./data_original/data/test_imgs/BC_01_0488.png
./data_original/data/test_imgs/BC_01_0507.png
./data_original/data/test_imgs/BC_01_0511.png
./data_original/data/test_imgs/BC_01_0526.png
./data_original/data/test_imgs/BC_01_0632.png
./data_original/data/test_imgs/BC_01_0635.png
./data_original/data/test_imgs/BC_01_0712.png
./data_original/data/test_imgs/BC_01_0715.png
./data_original/data/test_imgs/BC_01_0721.png
./data_original/data/test_imgs/BC_01_0726.png
./data_original/data/test_imgs/BC_01_0790.png
./data_original/data/test_imgs/BC_01_0793.png
./data_original/data/test_imgs/BC_01_0801.png


 12%|█▎        | 2/16 [00:17<02:04,  8.87s/it]

...

 88%|████████▊ | 14/16 [02:07<00:18,  9.38s/it]

./data_original/data/test_imgs/BC_01_3009.png
./data_original/data/test_imgs/BC_01_3022.png
./data_original/data/test_imgs/BC_01_3023.png
./data_original/data/test_imgs/BC_01_3024.png
./data_original/data/test_imgs/BC_01_3027.png
./data_original/data/test_imgs/BC_01_3029.png
./data_original/data/test_imgs/BC_01_3052.png
./data_original/data/test_imgs/BC_01_3055.png
./data_original/data/test_imgs/BC_01_3057.png
./data_original/data/test_imgs/BC_01_3059.png
./data_original/data/test_imgs/BC_01_3074.png
./data_original/data/test_imgs/BC_01_3082.png
./data_original/data/test_imgs/BC_01_3114.png
./data_original/data/test_imgs/BC_01_3124.png
./data_original/data/test_imgs/BC_01_3167.png
./data_original/data/test_imgs/BC_01_3201.png


 94%|█████████▍| 15/16 [02:16<00:09,  9.40s/it]

./data_original/data/test_imgs/BC_01_3205.png
./data_original/data/test_imgs/BC_01_3211.png
./data_original/data/test_imgs/BC_01_3256.png
./data_original/data/test_imgs/BC_01_3257.png
./data_original/data/test_imgs/BC_01_3261.png
./data_original/data/test_imgs/BC_01_3328.png
./data_original/data/test_imgs/BC_01_3404.png
./data_original/data/test_imgs/BC_01_3418.png
./data_original/data/test_imgs/BC_01_3438.png
./data_original/data/test_imgs/BC_01_3446.png


100%|██████████| 16/16 [02:20<00:00,  8.79s/it]


In [ ]:
preds

### Submission

In [ ]:
submit = pd.read_csv("./data_original/data/sample_submission.csv")

submit["N_category"] = preds
submit.to_csv("./submit/submission.csv", index = False)